In [1]:
import re
import requests

In [2]:
def extract_imdb_id(imdb_url):
    """
    Function to extract IMDb ID from link
    """
    try:
        pattern = r'tt\d{5,9}'  # "tt" + from 5 to 9 numbers (IMDb ID length is usually 9 numbers)
        match = re.search(pattern, imdb_url)
        return match.group()
    except StopIteration:
        return "Unable to select movie id. Load another link"

In [3]:
TMDB_API_KEY = "4ef7fb9aeb9935e011c8af3c495cdd6c"

def get_movie_from_imdb(imdb_url):
    imdb_id = extract_imdb_id(imdb_url)
    if not imdb_id:
        print("error.Invalid IMDb URL")
        
    tmdb_url = f"https://api.themoviedb.org/3/find/{imdb_id}"
    params = {
        "api_key": TMDB_API_KEY,
        "external_source": "imdb_id"
    }
    
    response = requests.get(tmdb_url, params=params)
    if response.status_code != 200:
        return {"error": f"Failed to connect to TMDb API. Status code: {response.status_code}"}
    
    data = response.json()
    print(data)
    if not data.get('movie_results') and not data.get('tv_results'):
        return {"error": "No movie or TV show found for this IMDb ID"}
    
    if data.get('movie_results'):
        movie = data['movie_results'][0]
        return {
            "type": "movie",
            "title": movie.get("title"),
            "overview": movie.get("overview"),
            "release_date": movie.get("release_date"),
            "vote_average": movie.get("vote_average"),
            "poster_path": f"https://image.tmdb.org/t/p/w500{movie.get('poster_path')}" if movie.get("poster_path") else None
        }
    elif data.get('tv_results'):
        tv_show = data['tv_results'][0]
        return {
            "type": "tv_show",
            "title": tv_show.get("name"),
            "overview": tv_show.get("overview"),
            "first_air_date": tv_show.get("first_air_date"),
            "vote_average": tv_show.get("vote_average"),
            "poster_path": f"https://image.tmdb.org/t/p/w500{tv_show.get('poster_path')}" if tv_show.get("poster_path") else None
        }


imdb_url = "https://www.imdb.com/title/tt0066921/?ref_=nv_sr_srsg_1_tt_7_nm_0_in_0_q_%25D0%25B7%25D0%25B0%25D0%25B2%25D0%25BE%25D0%25B4%25D0%25BD%25D0%25BE%25D0%25"
movie_data = get_movie_from_imdb(imdb_url)

if "error" in movie_data:
    print("Ошибка:", movie_data["error"])
else:
    print("Тип:", "Фильм" if movie_data["type"] == "movie" else "Series")
    print("Название:", movie_data["title"])
    print("Описание:", movie_data["overview"])
    print("Дата выхода:", movie_data.get("release_date") or movie_data.get("first_air_date"))
    print("Рейтинг:", movie_data["vote_average"])
    print("Постер:", movie_data["poster_path"])

{'movie_results': [{'backdrop_path': '/nLFxvLokHe3bQmrmAfljIfax2jQ.jpg', 'id': 185, 'title': 'A Clockwork Orange', 'original_title': 'A Clockwork Orange', 'overview': "In a near-future Britain, young Alexander DeLarge and his pals get their kicks beating and raping anyone they please. When not destroying the lives of others, Alex swoons to the music of Beethoven. The state, eager to crack down on juvenile crime, gives an incarcerated Alex the option to undergo an invasive procedure that'll rob him of all personal agency. In a time when conscience is a commodity, can Alex change his tune?", 'poster_path': '/4sHeTAp65WrSSuc05nRBKddhBxO.jpg', 'media_type': 'movie', 'adult': False, 'original_language': 'en', 'genre_ids': [878, 80], 'popularity': 39.763, 'release_date': '1971-12-19', 'video': False, 'vote_average': 8.2, 'vote_count': 12938}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': []}
Тип: Фильм
Название: A Clockwork Orange
Описание: In a near